# Exercise 2: Plan-Execute + Reflection Hybrid

In [1]:
# Imports
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
from pydantic import BaseModel, Field

import os
import operator
print("All imports successful")

All imports successful


In [2]:
load_dotenv()
api_key = os.getenv("paid_api2")

if not api_key:
    raise ValueError("API_Key not found. Please set it in your .env file")
print("API key loaded")

API key loaded


In [3]:
## Initialize LLM
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.5,
    api_key = api_key
)
print(f"LLM initialized: {llm.model_name}")

LLM initialized: gpt-4o-mini


## Plan-Execute Pattern

In [4]:
#Customized state for plan-execute
class HybridState(TypedDict):
    """State for hybrid pattern combining plan-execute and reflection pattern."""
    #Plan-execute
    input: str 
    plan: list[str]
    current_step: int
    results: Annotated[list[str], operator.add] #Results from each step
    
    #Reflection pattern
    iterations: str
    draft: str
    critique: str
    final_output: str

MAX_ITERATIONS =2
print("Hybrid state defined")

Hybrid state defined


In [5]:
# Node 1: Planner
def planner(state: HybridState) -> dict:
    """Create a step-by-step plan."""
    prompt = f"""Create a step-by-step plan for this task:

Task: {state['input']}

Return a numbered list of 3 concrete steps. Keep it simple."""
    
    response = llm.invoke([HumanMessage(content=prompt)])

    # Parse steps (simple parsing)
    lines = response.content.split('\n')
    steps = [line.strip() for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]

    # print(lines)
    print(f"\nPLAN CREATED:")
    for step in steps:
        print(f"    {step}")
    print()

    return {"plan": steps, "current_step": 0, "results": []}

# Node 2: Executor
def executor(state: HybridState) -> dict:
    """Execute current step."""
    if state["current_step"] >= len(state["plan"]):
        #All steps done
        return {}
    
    current_step = state["plan"][state["current_step"]] #maps current step to a planned task

    print(f"Executing: {current_step}")

    # Execute step (simplified - just use LLM)
    prompt = f"""Previous results: {state.get('results', [])}\n\nExecute this step: {current_step}"""
    response = llm.invoke([HumanMessage(content=prompt)])

    result = f"Step {state['current_step']+1} result: {response.content}"
    # print(result)
    print(f"✓ Done\n")

    return {
        "results": [result],
        "current_step": state["current_step"]+1
    }

# print("Plan-Execute nodes defined")

#Node Generator
def generator(state: HybridState) -> dict:
    """Synthesizes all execution results into a final draft."""
    print("\nSynthesizing execution results into a draft...")

    prompt = f"""Synthesize these research results into a final answer for the user.
Original Task: {state['input']}
Research Results: {state['results']}
    
Ensure the response is comprehensive, direct. and beginner-friendly."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {
        "draft": response.content,
        "iterations": 0
    }

#Node Critic
def critic(state: HybridState) -> dict:
    """Evaluates the draft and suggests improvements."""
    print("Critiquing draft for quality and beginner-friendliness...")

    prompt = f"""Evaluate this draft against the original task.
Task: {state['input']}
Draft: {state['draft']}
    
Critique it for:
1. It's research quality
2. Tone (must be beginner-friendly)
3. Clarity and engagement
    
If it is excellent, say 'APPROVED: explanation'. 
Otherwise, list specific improvements needed."""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {
        "critique": response.content,
        "iterations": state["iterations"] + 1
    }

# Node refiner
def refiner(state: HybridState) -> dict:
    """Refines the draft based on the critic's feedback."""
    print(f"Refining draft (Iteration {state['iterations']})...")

    prompt = f"""Refine the draft below using the provided critique.
Original Task: {state['input']}
Current Draft: {state['draft']}
Critique: {state['critique']}
    
Provide an improved, high-quality version."""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content}

def finalizer(state: HybridState) -> dict:
    """Returns the final refiined output."""
    print("\nHybrid process complete!")
    return {
        "final_output": state["draft"]}

print("All nodes defined")




All nodes defined


### Routing Function

In [6]:
def should_continue_execution(state: HybridState) -> Literal["executor", "generator"]:
    """Decide if there are more steps to be executed."""
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "generator"

def should_reflect_again(state: HybridState) -> Literal["refiner", "finalizer"]:
    """Decide if further refinement is needed."""
    if "APPROVED" in state.get("critique", "").upper():
        return "finalizer"
    
    if state["iterations"] >= MAX_ITERATIONS:
        print(f"Max reflections ({MAX_ITERATIONS})reached.")
        return "finalizer"
    
    return "refiner"

print("Routing functions defined")

Routing functions defined


### Hybrid Graph

In [7]:
hybrid_builder = StateGraph(HybridState)

# Add nodes
hybrid_builder.add_node("planner", planner)
hybrid_builder.add_node("executor", executor)
hybrid_builder.add_node("generator", generator)
hybrid_builder.add_node("critic", critic)
hybrid_builder.add_node("refiner", refiner)
hybrid_builder.add_node("finalizer", finalizer)

#Add edges
hybrid_builder.add_edge(START, "planner")
hybrid_builder.add_edge("planner", "executor")

hybrid_builder.add_conditional_edges(
    "executor", should_continue_execution,
    {"executor": "executor", "generator": "generator"}
)

hybrid_builder.add_edge("generator", "critic")
hybrid_builder.add_conditional_edges(
    "critic", should_reflect_again,
    {"refiner": "refiner", "finalizer": "finalizer"}
)
hybrid_builder.add_edge("refiner", "critic")
hybrid_builder.add_edge("finalizer", END)

hybrid_agent = hybrid_builder.compile()

print("Hybrid Graph compiled")

Hybrid Graph compiled


In [8]:
query = "Research the benefits of Python programming, create a summary, and make it beginner-friendly"

result = hybrid_agent.invoke({
    "input": query,
    "plan": [],
    "current_step": 0,
    "results": [],
    "iterations": 0,
    "critique": "",
    "draft": ""
    
})

print(f"\n{'='*70}")
print("📊 FINAL OUTPUT:")
print(f"{'='*70}")
print(result["final_output"])
print(f"{'='*70}\n")


PLAN CREATED:
    1. **Conduct Research on Python Benefits**:
    2. **Summarize Key Points**:
    3. **Create a Beginner-Friendly Summary**:

Executing: 1. **Conduct Research on Python Benefits**:
✓ Done

Executing: 2. **Summarize Key Points**:
✓ Done

Executing: 3. **Create a Beginner-Friendly Summary**:
✓ Done


Synthesizing execution results into a draft...
Critiquing draft for quality and beginner-friendliness...

Hybrid process complete!

📊 FINAL OUTPUT:
**Why Choose Python?**

Python is a fantastic programming language, especially if you're just starting out. Here’s a beginner-friendly summary of its many benefits:

1. **Easy to Learn**: Python has a simple and clear syntax, which makes it perfect for beginners. You can grasp programming concepts quickly without getting bogged down by complicated rules.

2. **Versatile**: You can use Python for a wide range of applications, including building websites, analyzing data, creating machine learning models, and automating tasks. This